In [1]:
import numpy as np
import matplotlib.pyplot as plt
from BCS_casadi import BCS_model
from plot_result import PlotResult
from bcs_envelope import BcsEnvelope
from nmpc_class import NMPC
import seaborn as sns
sns.set_theme()
import pandas as pd
from tictoc import tic, toc



# steady-state conditions
xss = np.vstack([8311024.82175957, 2990109.06207437,
                0.00995042241351780, 50., 50.])
nx = 5; nu = 2 ;ny = 2

uss = np.vstack([50., 50.])
yss = np.vstack([6000142.88550200, 592.126490003812])
# Controller parameters
Hp = 10  # prediction horizon 
Hc = 2  # control horizon
Ts = 2  # sampling time
# --------------------------------------------------------------------------
# constraints on inputs
# --------------------------------------------------------------------------
umin = np.vstack([35, 0])    # lower bounds of inputs
umax = np.vstack([65, 100])    # upper bounds of inputs
# maximum variation of input moves: [0.5 Hz/s; 0.5 #/s]
dumax = Ts*np.vstack([0.5, 0.5])
# ----------------------------------------------
# ------Normalização através dos pesos----------
q = np.hstack([100, 1]) / (yss**2)  # weights on controlled variables
q = np.hstack([1e6, 1e8]) / (yss**2)  # weights on controlled variables
r = np.array([100, 1]) / (uss.T**2)  # weights on control actions

qu = 1000 / (uss[1]**2)

print("Instancia BCS")
bcs_init = [nu, nx, ny, Ts, umin, umax, dumax]
nmpc = NMPC(Hp, Hc, q, r, qu, bcs_init)
bcs = nmpc.bcs

Dados carregados
Instancia BCS


In [2]:
# --------------------------------------------------------------------------
# Initial condition (preferred steady-state)
# --------------------------------------------------------------------------
uk_1 = np.vstack([50., 50.])  # manipulated variable
print("Initial states")
print("Pbh, Pwh, q, df, dzc")
x0 = np.vstack([8311024.82175957, 2990109.06207437,
               0.00995042241351780, 50., 50.])  # state variables
print("Initial control")
print("f, zc")


utg = 60   # target na choke
pm = 2e6   # pressão da manifold
# ymax[0,0] = yss[0]; # Pressao de intake
# Regiao de operação
hlim = bcs.envelope.Hlim(x0[2]*3600)
ymin = np.vstack([yss[0], min(hlim)])  # Pressao de intake e  Downtrhust
ymax = np.vstack([xss[0], max(hlim)])  # Pressao de intake e  Uptrhust
# ymin(2,1) = min(hlim); # Downtrhust
# ymax(2,1) = max(hlim); # Uptrhust
xpk = xss
ysp = yss
# Simulation Loop -------------------------------- ------------------------------
tsim = 50    # seconds
nsim = int(tsim/Ts)   # number of steps
uk = np.zeros((bcs.nu, int(nsim)))
rows=[]

Vruido = ((0.01/3)*np.diag(yss[:, 0]))**2
Du = np.zeros((nmpc.Hc*bcs.nu, 1))
# # Parameters: initial states, du,utg, u0,ysp
P = np.vstack([x0, uk_1, Du, utg])

Yk = yss
Xk = xss.reshape((xss.shape[0], 1))
Ysp = yss
YLim = np.vstack([ymax[1], ymin[1]])

Initial states
Pbh, Pwh, q, df, dzc
Initial control
f, zc


In [3]:
for k in range(nsim):
    print("Iteração:", k)
    tsim = k*Ts
    ymin[0,0] = 8.8e6
    # changes on set-points Pintake
    if tsim==50:
        ymin[0,0] = 6e6
    elif tsim==100:
        ymin[0,0] = 4.2e6


    ymax[0, 0] = ymin[0, 0]
    # elif k==200:
    #    ymin[0,0] = 4.2e6
    # else:
    #     bcs.pm = 8e5
    # if k==20:
    #     utg=80
    #     #uk_1[0,0]=55
    # if k==int(50/Ts):
    #     utg=90
   

    # #ymin(1,1) = yss(1);    # Pressao de intake
    # ymax[0,0] = ymin[0,0]; # Pressao de intake

    # #ymin[0,0] = 4e6;
    # ## Limite Up e Downthrust
    # hlim = bcs.envelope.Hlim(xpk[2]*3600)
    ymin[1, 0] = min(hlim)
    ymax[1, 0] = max(hlim)

    tic()
    
    Du, ysp, sol, opti = nmpc.nmpc_solver(P, ymin, ymax)
    
    #W=sol.value_variables()
    if sol.stats()['success']:
        flag=0
    else:
        flag=1
    
    #print(sol.value_variables())
    
    
    cost=sol.stats()["iterations"]["obj"][-1]
    n_iter=sol.stats()["iter_count"]
    rows.append([k,toc(),cost,n_iter,flag])
    uk[:, k:k+1] = uk_1 + Du[:nmpc.Hc, :]
    uk_1 += Du[:nmpc.Hc, :]  # optimal input at time step k
    #update input vector with the states and Du
    P = np.vstack([x0, uk_1, Du, utg])
    

    # Plant
    xpk = bcs.integrator_ode(x0, uk_1)
    #print(xpk)
    #  Nominal Model
    x0 = xpk
    ypk = bcs.eq_medicao(x0)
    ruido = 5
    # +np.random.multivariate_normal((0,0), Vruido).reshape((bcs.ny,1)) # ruido -> 3x a 5x
    ypk = ypk
    Yk = np.concatenate((Yk, ypk), axis=1)
    Ysp = np.concatenate((Ysp, ysp), axis=1)
    YLim = np.concatenate((YLim, np.vstack([ymax[1], ymin[1]])), axis=1)
    Xk = np.concatenate((Xk, x0.reshape((x0.shape[0], 1))), axis=1)

Iteração: 0
[[0.656882, 0.589818, 0.41643, 50, 50]]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   3.08ms ( 57.02us)   3.23ms ( 59.79us)        54
       nlp_g  | 526.81ms (  9.76ms) 523.16ms (  9.69ms)        54
  nlp_grad_f  |   2.70ms (112.50us)   2.72ms (113.51us)        24
  nlp_hess_l  |   4.11 s (171.18ms)   4.08 s (170.10ms)        24
   nlp_jac_g  |   1.79 s ( 44.78ms)   1.78 s ( 44.48ms)        40
       total  |   6.47 s (  6.47 s)   6.43 s (  6.43 s)         1


RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:159:
.../casadi/core/optistack_internal.cpp:999: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Restoration_Failed'

In [6]:
try:
    plt.figure(1)
    nmpc.opti.callback(lambda i: plt.plot(nmpc.opti.debug.value(x0[0])))
    Du, ysp, sol, opti = nmpc.nmpc_solver(P, ymin, ymax)
except RuntimeError:
    print("Erro no solver")
    #if sol.stats()['success']==1:
    nmpc.opti.debug.show_infeasibilities()
    plt.show()
    

    # sol = opti.solve()
    



[[0.656882, 0.589818, 0.41643, 50, 50]]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   2.94ms ( 54.43us)   3.07ms ( 56.89us)        54
       nlp_g  | 528.09ms (  9.78ms) 525.96ms (  9.74ms)        54
  nlp_grad_f  |   2.75ms (114.50us)   2.82ms (117.32us)        24
  nlp_hess_l  |   4.16 s (173.32ms)   4.14 s (172.63ms)        24
   nlp_jac_g  |   1.81 s ( 45.17ms)   1.80 s ( 44.98ms)        40
       total  |   6.54 s (  6.54 s)   6.51 s (  6.51 s)         1
Erro no solver
Violated constraints (tol 0), in order of declaration:
------- i = 0/83 ------ 
0.656882 <= 0.880871 <= 0.656882 (viol 0.223989)
Opti constraint of shape 5x1, defined at /home/taniel/Projetos_dev/NMPC_PINN/simulador_BCS_pavlov/nmpc_class.py:56 in nmpc_solver
  eps = 0.05
At nonzero 0.
------- i = 1/83 ------ 
0.589818 <= 0.589818 <= 0.589818 (viol 1.44202e-09)
Opti constraint of shape 5x1, defined at /home/taniel/Projetos_dev/NMPC_PINN/simulador_BCS_pavlov/nmpc_class.py:56 in nm

<Figure size 640x480 with 0 Axes>

In [10]:
18*Ts/60

0.6

|> 0 : 6.1578825035511181e+06
|> 0 : 5.6432914969577769e+02
|> 0 : 6.1665887813196341e+06
|> 0 : 5.6854323693339791e+02
|> 0 : 6.1768888896247149e+06
|> 0 : 5.7006114789511173e+02
|> 0 : 6.1653464429517444e+06
|> 0 : 5.6838933255619440e+02
|> 0 : 6.1665698849928938e+06
|> 0 : 5.6787924465096319e+02
|> 0 : 6.1673921102962447e+06
|> 0 : 5.6768056544316960e+02
|> 0 : 6.1677624301414285e+06
|> 0 : 5.6760754255574932e+02
|> 0 : 6.1679086048238641e+06
|> 0 : 5.6758062169037328e+02
|> 0 : 6.1679640401977291e+06
|> 0 : 5.6757071714627523e+02
|> 0 : 6.1679847348534577e+06
|> 0 : 5.6756707381189483e+02
Violated constraints (tol 0), in order of declaration:
------- i = 0/78 ------ 
0 <= 0.001072919571519892 <= 0 (viol 0.001072919571519892)
Opti constraint of shape 5x1, defined at /home/taniel/Projetos_dev/NMPC_PINN/simulador_BCS_pavlov/nmpc_class.py:65 in nmpc_solver
  opti.subject_to(st_next == self.bcs.F(x0=st, p=u)['xf'])
At nonzero 0.
------- i = 1/78 ------ 
0 <= 0.002409593687349609 <= 0 (v

In [ ]:
res = pd.DataFrame(rows, columns=['k','t_calc','cost','n_iter','flag'])    
#define plotting region (2 rows, 2 columns)
fig_res, axes = plt.subplots(2, 2)
sns.scatterplot(data=res, x='k',y='t_calc',ax=axes[0,0])
sns.scatterplot(data=res, x='k',y='cost',ax=axes[0,1])
sns.scatterplot(data=res, x='k',y='n_iter',ax=axes[1,0])
sns.scatterplot(data=res, x='k',y='flag',ax=axes[1,1])


#sns.relplot(data=res, kind="line", x="k", y="t_calc", col="t_calc")
# sns.lineplot(data=res, palette="tab10", linewidth=2.5)
grafico = PlotResult()
grafico.plot_resultado(Xk, uk)
grafico.plot_y(Ysp, Yk, YLim)
bcs.envelope.size_env = (4, 4)
bcs.envelope.grafico_envelope(Xk, Yk)
plt.show()